In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip intall folium

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "intall" - maybe you meant "install"



In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip intall seaborn

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "intall" - maybe you meant "install"



In [6]:
pip intall geopy

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "intall" - maybe you meant "install"



In [7]:
pip intall scipy

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "intall" - maybe you meant "install"



In [8]:
pip intall scikit-learn

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "intall" - maybe you meant "install"



In [9]:
pip install tabulate

In [10]:
pip install Django==4.1.5

Note: you may need to restart the kernel to use updated packages.


In [15]:
import geopy
from pandas import json_normalize
import folium
from geopy.geocoders import Nominatim 
import requests

city = "chennai"
## get location
locator = geopy.geocoders.Nominatim(user_agent="MyCoder")
location = locator.geocode(city)
print(location)
latpoint=location.latitude
longpoint=location.longitude
## keep latitude and longitude only
location = [location.latitude, location.longitude]
print("[lat, long]:", location)
print(latpoint,longpoint)

Chennai, Chennai District, Tamil Nadu, 600001, India
[lat, long]: [13.0836939, 80.270186]
13.0836939 80.270186


In [17]:
import json  
import pandas as pd  
from pandas import json_normalize  
import requests
from tabulate import tabulate
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import folium



#Fetching data form HERE API for IIT Bombay
latitude=latpoint
longitude=longpoint

url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r=10000&limit=100&q=apartment&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(latpoint,longpoint)
data = requests.get(url).json()
d=json_normalize(data['items'])
d.to_csv('D:/jupyter/apartment.csv')



#Cleaning API data
d2=d[['title','address.label','distance','access','position.lat','position.lng','address.postalCode','id']]
d2.to_csv('D:/jupyter/cleaned_apartment.csv')



#Counting no. of cafes, department stores and gyms near apartments around IIT Bombay
df_final=d2[['position.lat','position.lng']]

CafeList=[]
DepList=[]
GymList=[]
latitudes = list(d2['position.lat'])
longitudes = list( d2['position.lng'])
for lat, lng in zip(latitudes, longitudes):    
    radius = '1000' #Set the radius to 1000 metres
    latitude=lat
    longitude=lng
    
    search_query = 'hotels' #Search for any cafes
    url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(latitude, longitude, radius, search_query)
    results = requests.get(url).json()
    venues=json_normalize(results['items'])
    CafeList.append(venues['title'].count())
	
    search_query = 'gym' #Search for any gyms
    url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(latitude, longitude, radius, search_query)
    results = requests.get(url).json()
    venues=json_normalize(results['items'])
    GymList.append(venues['title'].count())

    search_query = 'department-store' #search for supermarkets
    url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(latitude, longitude, radius, search_query)
    results = requests.get(url).json()
    venues=json_normalize(results['items'])
    DepList.append(venues['title'].count())

df_final['Cafes'] = CafeList
df_final['Department Stores'] = DepList
df_final['Gyms'] = GymList

print(tabulate(df_final,headers='keys',tablefmt='github'))



#Run K-means clustering on dataframe
kclusters = 4

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_final)
df_final['Cluster']=kmeans.labels_
df_final['Cluster']=df_final['Cluster'].apply(str)

print(tabulate(df_final,headers='keys',tablefmt='github'))



#Plotting clustered locations on map using Folium

#define coordinates of the user location
map_bom=folium.Map(location=[latpoint,longpoint],zoom_start=12)

# instantiate a feature group for the incidents in the dataframe
locations = folium.map.FeatureGroup()

# set color scheme for the clusters
def color_producer(cluster):
    if cluster=='0':
        return 'green'
    elif cluster=='1':
        return 'blue'
    elif cluster=='2':
        return 'yellow'
    else:
        return 'red'

latitudes = list(df_final['position.lat'])
longitudes = list(df_final['position.lng'])
labels = list(df_final['Cluster'])
names=list(d2['title'])
for lat, lng, label,names in zip(latitudes, longitudes, labels,names):
    folium.CircleMarker(
            [lat,lng],
            fill=True,
            fill_opacity=1,
            popup=folium.Popup(names, max_width = 300),
            radius=5,
            color=color_producer(label)
        ).add_to(map_bom)

# add locations to map
map_bom.add_child(locations)
folium.Marker([latpoint,longpoint],popup='IIT Bombay').add_to(map_bom)

#saving the map 
map_bom.save("map-IITBombay.html")
map_bom

C:\Users\mukal\AppData\Local\Temp\ipykernel_42216\2619175192.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  d=json_normalize(data['items'])
C:\Users\mukal\AppData\Local\Temp\ipykernel_42216\2619175192.py:47: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  venues=json_normalize(results['items'])
C:\Users\mukal\AppData\Local\Temp\ipykernel_42216\2619175192.py:53: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  venues=json_normalize(results['items'])
C:\Users\mukal\AppData\Local\Temp\ipykernel_42216\2619175192.py:59: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  venues=json_normalize(results['items'])
C:\Users\mukal\AppData\Local\Temp\ipykernel_42216\2619175192.py:47: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  venues=json_normalize(r

|    |   position.lat |   position.lng |   Cafes |   Department Stores |   Gyms |
|----|----------------|----------------|---------|---------------------|--------|
|  0 |        13.0814 |        80.2679 |      20 |                  10 |     20 |
|  1 |        13.0837 |        80.2766 |      20 |                  19 |     15 |
|  2 |        13.0896 |        80.2679 |      20 |                  11 |     20 |
|  3 |        13.0838 |        80.2768 |      20 |                  20 |     15 |
|  4 |        13.0844 |        80.2769 |      20 |                  20 |     16 |
|  5 |        13.0897 |        80.2674 |      20 |                  11 |     20 |
|  6 |        13.09   |        80.2666 |      20 |                  10 |     20 |
|  7 |        13.09   |        80.2662 |      20 |                  11 |     20 |
|  8 |        13.091  |        80.2626 |      20 |                  17 |     20 |
|  9 |        13.0916 |        80.263  |      20 |                  15 |     20 |
| 10 |        13